# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [4]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum, max as Fmax
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline

import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Spark") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [6]:
# Reading the dataset
user_log = spark.read.json("mini_sparkify_event_data.json")

In [7]:
# Printing out the columns and their types
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
# A brief description about userId column
user_log.describe('userId').show()

+-------+-----------------+
|summary|           userId|
+-------+-----------------+
|  count|           286500|
|   mean|59682.02278593872|
| stddev|109091.9499991047|
|    min|                 |
|    max|               99|
+-------+-----------------+



In [9]:
# Printing the first line
user_log.take(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')]

In [10]:
# The number of lines
user_log.count()

286500

In [11]:
# number of users with no ids
user_log.filter(user_log['userId']=='').count()

8346

In [12]:
# Deleting users with no ids
df_cleaned = user_log.filter(user_log['userId']!='')

In [13]:
# The type of the column type, it shouldn't be a string
df_cleaned.describe('ts')

DataFrame[summary: string, ts: string]

In [14]:
# Create a user defined function for formating the timestamp
get_time = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%Y-%m-%d %H:%M:%S"))

#Apply the udf on the ts column to transform the type
df_cleaned = df_cleaned.withColumn("time", get_time(df_cleaned.ts))

In [15]:
# Attributing the results to a new variable
df_cleaned = df_cleaned.dropDuplicates()

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [16]:
# Creating a view to use with spark sql
df_cleaned.createOrReplaceTempView("Sparkify")

In [17]:
# The description of the columns
df_cleaned.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- time: string (nullable = true)



In [18]:
# Informations about the column auth
df_cleaned.describe('auth').show()

+-------+---------+
|summary|     auth|
+-------+---------+
|  count|   278154|
|   mean|     null|
| stddev|     null|
|    min|Cancelled|
|    max|Logged In|
+-------+---------+



In [19]:
# Number of users in each type of authentication
spark.sql(
    '''
    SELECT auth, COUNT(DISTINCT userId) as users from Sparkify 
        GROUP BY auth
    '''
).show()

+---------+-----+
|     auth|users|
+---------+-----+
|Cancelled|   52|
|Logged In|  225|
+---------+-----+



In [20]:
# Number of users in each gender
spark.sql('''
        SELECT gender,COUNT(DISTINCT userId) AS user_counts
            FROM sparkify
                GROUP BY gender
                    ORDER BY user_counts DESC
''').show()

+------+-----------+
|gender|user_counts|
+------+-----------+
|     M|        121|
|     F|        104|
+------+-----------+



In [21]:
# Number of users in each type of account's level
spark.sql('''
        SELECT level,COUNT(DISTINCT userId) AS user_counts
        FROM Sparkify
        GROUP BY level
        ORDER BY user_counts DESC
''').show()

+-----+-----------+
|level|user_counts|
+-----+-----------+
| free|        195|
| paid|        165|
+-----+-----------+



In [22]:
# Number of users in each type of page
spark.sql('''
        SELECT page,COUNT(userId) AS user_counts
        FROM Sparkify
        GROUP BY page
        ORDER BY user_counts DESC
''').show()

+--------------------+-----------+
|                page|user_counts|
+--------------------+-----------+
|            NextSong|     228108|
|           Thumbs Up|      12551|
|                Home|      10082|
|     Add to Playlist|       6526|
|          Add Friend|       4277|
|         Roll Advert|       3933|
|              Logout|       3226|
|         Thumbs Down|       2546|
|           Downgrade|       2055|
|            Settings|       1514|
|                Help|       1454|
|             Upgrade|        499|
|               About|        495|
|       Save Settings|        310|
|               Error|        252|
|      Submit Upgrade|        159|
|    Submit Downgrade|         63|
|              Cancel|         52|
|Cancellation Conf...|         52|
+--------------------+-----------+



In [23]:
# The function to map the values, attributing the value 1 if the customer has cancelled the services, and 0 otherwise.
mapper = udf(lambda x : 1 if x=='Cancellation Confirmation' else 0, IntegerType())

In [24]:
# Applying the function
df_cleaned = df_cleaned.withColumn('Churn_state',mapper('page'))
userwindow = Window.partitionBy('userId').rangeBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df_cleaned_cancel = df_cleaned.withColumn('Churn',Fmax('Churn_state').over(userwindow))

In [25]:
# View the first line
df_cleaned_cancel.take(1)

[Row(artist='Imogen Heap', auth='Logged In', firstName='Darianna', gender='F', itemInSession=19, lastName='Carpenter', length=249.05098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=113, song='Have You Got It In You?', status=200, ts=1539701061000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', time='2018-10-16 14:44:21', Churn_state=0, Churn=0)]

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [28]:
# Creating a function to encode the column
def label_encoding(col_name):
    '''
    transform categorical items to number
    '''
    temp = df_cleaned_cancel.select([col_name]).dropDuplicates().toPandas()
#     print(temp)
    label_dict = {val:str(idx) for idx,val in enumerate(temp[col_name].tolist())}
    result = df_cleaned_cancel.dropDuplicates(['userId']).select(['userId',col_name]).replace(label_dict,subset=col_name)
    return result

In [29]:
# Using the function above inside this function, to return the joins of the columns
def get_categorical_features():
    '''
    join all categorical features together
    '''
    feature_gender = label_encoding('gender')
    feature_level = label_encoding('level')
    feature_page = label_encoding('page')
    feature_auth = label_encoding('auth')
    
    result = feature_gender.join(feature_level,on='userId',how='inner')\
                           .join(feature_page, on='userId', how='inner')\
                           .join(feature_auth, on='userId', how='inner')
    return result

In [30]:
# Getting the categorical features and saving them in a variable
features = get_categorical_features()

In [32]:
# Filtering the original dataframe with only certain column
df_filtered = df_cleaned_cancel.select(['userId','length', 'ts', 'churn'])

In [33]:
# Join of the features and dataframe and removing the user id, because I'm not going to use it in machine learning
full_df = features.join(df_filtered,on='userId',how='inner')
full_df = full_df.drop('userId')

In [34]:
# Converting the type of the columns to integer
full_df = full_df.withColumn("gender",col("gender").cast(IntegerType()))\
    .withColumn("level",col("level").cast(IntegerType()))\
    .withColumn("page",col("page").cast(IntegerType()))\
    .withColumn('auth', col('auth').cast(IntegerType()))

In [35]:
# Removing missing values
full_df = full_df.dropna()

In [36]:
# The first row
full_df.take(1)

[Row(gender=0, level=0, page=13, auth=1, length=249.05098, ts=1539701061000, churn=0)]

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [37]:
# Splitting the dataset in train and test
train, test = full_df.randomSplit([0.7, 0.3], seed=0)

In [42]:
# A filter, with the columns that I will use to input
x = [col for col in full_df.columns if col!='churn']
x

['gender', 'level', 'page', 'auth', 'length', 'ts']

In [43]:
# A funtion to build the model
def build_model(classifier, param):
    '''
    Builds the model train
    '''
    # The inputs e output
    assembler = VectorAssembler(inputCols=x, outputCol="features")
    
    # Using the Min Max Scaler function
    scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
    
    # Putting all in a pipeline
    pipeline = Pipeline(stages=[assembler, scaler, classifier])
    
    # Machine Learning Model
    model = CrossValidator(
        estimator=pipeline,
        estimatorParamMaps=param,
        evaluator=MulticlassClassificationEvaluator(labelCol='churn', metricName='f1'),
        numFolds=5,
    )
    return model

In [44]:
# Model
lr = LogisticRegression(featuresCol="scaled_features", labelCol="churn")

# param
param = ParamGridBuilder().build()

# Training and predicting
model = build_model(lr, param)
fit_model = model.fit(train)
pred = fit_model.transform(test)

# The results
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="churn")
f1_score = evaluator.evaluate(pred, {evaluator.metricName: "f1"})
print("f1: {}".format(f1_score))

f1: 0.784406692582004


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.